# 🤟 엘리스 AI 4기 3차 프로젝트 - 5팀 수화 인식 모델 학습 및 평가 코드

# 1. Import and Install Dependencies

- mediapipe 라이브러리는 Python 3.7.x - 3.9.x 에서 사용하길 권장
- protobuf의 경우 최신 버전은 4.x 이나 mediapipe를 사용하기 위해서는 3.20.1 이하 버전 설치
- 주요 라이브러리별 버전
```
# Python 3.9.0
opencv-python==4.6.0.66
numpy==1.22.4
matplotlib==3.5.2
mediapipe==0.8.10
protobuf==3.19.4
tensorboard==2.9.1
tensorflow==2.9.1
```

In [25]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                              ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection
- 데이터 구축을 위한 디렉터리 구조 잡기

데이터셋 폴더 아래와 같이 다시 지정할지 ? 아니면 dataset 폴더에 다 넣고 돌릴지 고민
- alphabet 데이터 ('./dataset/alphabet')
- words 데이터 ('./dataset/words')

In [51]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset') 

# dataset to use
alphabet = [chr(ord('a') + i) for i in range(26)]
words = []

# Actions that we try to detect
actions = np.array(alphabet)

# Thirty videos worth of data
no_sequences = 100

# Videos are going to be 30 frames in length
sequence_length = 30

In [10]:
# 알파벳별 Data 구축을 위해 디렉토리 생성
for action in actions: 
    for sequence in range(100, 100+no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

## 5-1. Make dataset for training

In [11]:
# cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        cap = cv2.VideoCapture(0)
        # Loop through sequences aka videos
        for sequence in range(60, 60+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                     
        cap.release()
        cv2.destroyAllWindows()

## 5-2. Make dataset for Test

In [26]:
# cap = cv2.VideoCapture(0)
# Set mediapipe model 
no_sequences_t = 1
DATA_PATH_FOR_TEST = os.path.join('test_dataset')

for action in actions: 
    for sequence in range(no_sequences_t):
        try: 
            os.makedirs(os.path.join(DATA_PATH_FOR_TEST, action, str(sequence)))
        except:
            pass
        
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        cap = cv2.VideoCapture(0)
        # Loop through sequences aka videos
        for sequence in range(no_sequences_t):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH_FOR_TEST, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
        cap.release()
        cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [28]:
label_map = {label:num for num, label in enumerate(actions)}

### 6-1. train data flatten & split

In [41]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [42]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [43]:
X.shape

(2600, 30, 258)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### 6-2. test data flatten

In [67]:
t_sequences, t_labels = [], []
no_sequences_t = 1 # 알파벳별 몇개씩 만들지
DATA_PATH_FOR_TEST = os.path.join('test_dataset')
sequence_length=30
# actions_t = ['j'] 특정 알파벳만 만드는 경우 주석 해제 후 아래 for문의 actions => actions_t로 변경
for action in actions:
    for sequence in range(no_sequences_t):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH_FOR_TEST, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        t_sequences.append(window)
        t_labels.append(label_map[action])

X_test2 = np.array(t_sequences)
y_test2 = to_categorical(t_labels).astype(int)

# 7. Build and Train LSTM Neural Network

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from datetime import datetime

In [46]:
# 날짜로 디렉토리 구분시 tensorboard 가 구분될 줄 알았으나 잘 안됨
# log_dir = os.path.join('Logs/', datetime.now().strftime("%Y%m%d-%H%M%S"))

# 기존 방식으로 사용하되 그래프 겹칠 때는 Logs 폴더 아예 삭제(아래 셀 실행)
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [45]:
# Logs 폴더 삭제하는 경우
# !echo y | rmdir /s Logs

Logs, 계속하시겠습니까(Y/N)? y 


Logs\train - 디렉터리가 비어 있지 않습니다.
Logs\VALIDA~1 - 디렉터리가 비어 있지 않습니다.


In [101]:
# 이전 학습 이력이 영향을 줄 수 있기 때문에 model 변수를 삭제 후 재할당
del model

In [102]:
# 모델 생성 - nicholas 코드 참고
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# Signsense 모델 참고
'''
dropout=0.2
TIMESTEPS = 30
frame_dim=258
model2 = Sequential()
model2.add(Input(shape = (TIMESTEPS, frame_dim)))
model2.add(LSTM(128, name="lstm1", dropout=dropout, return_sequences=True))
model2.add(LSTM(64, name="lstm2", dropout=dropout))
model2.add(Dense(13, activation="softmax"))
adam = Adam(lr = 0.0002)
model2.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=360, callbacks=[tb_callback])
'''

In [98]:
optimizer = Adam(learning_rate=0.01) 

In [103]:
# 모델 컴파일
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
# accuracy 떨어질 때 멈추게 하려고 했지만 숫자를 10 이하로 잡으면 일찍 끝나고,, 20으로 잡으면 한참을 떨어져야 멈추고..
# es_callback = tf.keras.callbacks.EarlyStopping(monitor='categorical_accuracy', patience=20)

In [104]:
# 모델 학습
# fit 파라미터 확인 -> https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

model.fit(
    X_train, 
    y_train, 
    epochs=200,
    initial_epoch=0,
    validation_data=(X_test, y_test),
    shuffle=True,
    callbacks=[tb_callback])

Epoch 1/200
78/78 [==============================] - 20s 171ms/step - loss: 3.2585 - categorical_accuracy: 0.0397 - val_loss: 3.2603 - val_categorical_accuracy: 0.0154
Epoch 2/200
78/78 [==============================] - 12s 148ms/step - loss: 3.2825 - categorical_accuracy: 0.0409 - val_loss: 3.2643 - val_categorical_accuracy: 0.0308
Epoch 3/200
78/78 [==============================] - 11s 145ms/step - loss: 3.3086 - categorical_accuracy: 0.0389 - val_loss: 3.2609 - val_categorical_accuracy: 0.0154
Epoch 4/200
78/78 [==============================] - 11s 145ms/step - loss: 3.2605 - categorical_accuracy: 0.0356 - val_loss: 3.2612 - val_categorical_accuracy: 0.0154
Epoch 5/200
78/78 [==============================] - 11s 146ms/step - loss: 3.2618 - categorical_accuracy: 0.0352 - val_loss: 3.2627 - val_categorical_accuracy: 0.0077
Epoch 6/200
78/78 [==============================] - 13s 163ms/step - loss: 3.2585 - categorical_accuracy: 0.0377 - val_loss: 3.2615 - val_categorical_accuracy:

78/78 [==============================] - 12s 154ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2694 - val_categorical_accuracy: 0.0077
Epoch 50/200
78/78 [==============================] - 12s 155ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2690 - val_categorical_accuracy: 0.0077
Epoch 51/200
78/78 [==============================] - 12s 153ms/step - loss: 3.2581 - categorical_accuracy: 0.0397 - val_loss: 3.2696 - val_categorical_accuracy: 0.0077
Epoch 52/200
78/78 [==============================] - 12s 153ms/step - loss: 3.2819 - categorical_accuracy: 0.0364 - val_loss: 3.3061 - val_categorical_accuracy: 0.0692
Epoch 53/200
78/78 [==============================] - 12s 152ms/step - loss: 3.2640 - categorical_accuracy: 0.0336 - val_loss: 3.2680 - val_categorical_accuracy: 0.0077
Epoch 54/200
78/78 [==============================] - 12s 151ms/step - loss: 3.2583 - categorical_accuracy: 0.0425 - val_loss: 3.2675 - val_categorical_accuracy: 0.0231

78/78 [==============================] - 16s 201ms/step - loss: 3.2582 - categorical_accuracy: 0.0377 - val_loss: 3.2699 - val_categorical_accuracy: 0.0231
Epoch 98/200
78/78 [==============================] - 12s 157ms/step - loss: 3.2582 - categorical_accuracy: 0.0385 - val_loss: 3.2701 - val_categorical_accuracy: 0.0077
Epoch 99/200
78/78 [==============================] - 12s 153ms/step - loss: 3.2581 - categorical_accuracy: 0.0348 - val_loss: 3.2698 - val_categorical_accuracy: 0.0077
Epoch 100/200
78/78 [==============================] - 12s 154ms/step - loss: 3.2582 - categorical_accuracy: 0.0381 - val_loss: 3.2697 - val_categorical_accuracy: 0.0077
Epoch 101/200
78/78 [==============================] - 12s 152ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2697 - val_categorical_accuracy: 0.0077
Epoch 102/200
78/78 [==============================] - 12s 155ms/step - loss: 3.2582 - categorical_accuracy: 0.0360 - val_loss: 3.2695 - val_categorical_accuracy: 0.0

78/78 [==============================] - 18s 235ms/step - loss: 3.2582 - categorical_accuracy: 0.0352 - val_loss: 3.2695 - val_categorical_accuracy: 0.0154
Epoch 146/200
78/78 [==============================] - 13s 161ms/step - loss: 3.2582 - categorical_accuracy: 0.0397 - val_loss: 3.2695 - val_categorical_accuracy: 0.0154
Epoch 147/200
78/78 [==============================] - 12s 160ms/step - loss: 3.2581 - categorical_accuracy: 0.0356 - val_loss: 3.2697 - val_categorical_accuracy: 0.0154
Epoch 148/200
78/78 [==============================] - 12s 160ms/step - loss: 3.2582 - categorical_accuracy: 0.0397 - val_loss: 3.2698 - val_categorical_accuracy: 0.0154
Epoch 149/200
78/78 [==============================] - 12s 158ms/step - loss: 3.2582 - categorical_accuracy: 0.0397 - val_loss: 3.2699 - val_categorical_accuracy: 0.0154
Epoch 150/200
78/78 [==============================] - 16s 209ms/step - loss: 3.2582 - categorical_accuracy: 0.0372 - val_loss: 3.2698 - val_categorical_accuracy: 0

78/78 [==============================] - 17s 217ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2688 - val_categorical_accuracy: 0.0077
Epoch 194/200
78/78 [==============================] - 17s 211ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2691 - val_categorical_accuracy: 0.0077
Epoch 195/200
78/78 [==============================] - 16s 203ms/step - loss: 3.2581 - categorical_accuracy: 0.0401 - val_loss: 3.2691 - val_categorical_accuracy: 0.0077
Epoch 196/200
78/78 [==============================] - 15s 186ms/step - loss: 3.2582 - categorical_accuracy: 0.0401 - val_loss: 3.2694 - val_categorical_accuracy: 0.0077
Epoch 197/200
78/78 [==============================] - 15s 187ms/step - loss: 3.2581 - categorical_accuracy: 0.0401 - val_loss: 3.2693 - val_categorical_accuracy: 0.0077
Epoch 198/200
78/78 [==============================] - 13s 173ms/step - loss: 3.2582 - categorical_accuracy: 0.0372 - val_loss: 3.2696 - val_categorical_accuracy: 0

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 26)                858       
                                                                 
Total params: 238,010
Trainable params: 238,010
Non-tr

In [39]:
# tensorboard 실행 => localhost(or vm domain):6006
# 다른 포트 사용시 맨 뒤에 --port=사용할 포트 번호
!tensorboard --logdir=./Logs

^C


# 8. Make Predictions for Test data
- 별도로 구축한 테스트 데이터(X_test2) 로 인식률 계산

In [90]:
res_test = model.predict(X_test2)
res_test.shape

1/1 [==============================] - 1s 521ms/step


(26, 26)

In [92]:
cnt = 0
for i in range(len(res_test)):
    if actions[np.argmax(res_test[i])] == actions[np.argmax(y_test2[i])]:
        cnt += 1
#     print(actions[np.argmax(res_test[i])])
(cnt / len(res)) * 100

61.53846153846154

# 9. Save Weights

In [93]:
model.save('action_hs_3.h5')

In [52]:
del model

In [19]:
model.load_weights('action_hs_2.h5')

# 10. Evaluation using Confusion Matrix and Accuracy
- scikit-learn의 multilabel_confusion_matrix, accuracy_score을 이용해서 정확도 계산

- multilabel_confusion_matrix(y_true, y_predict)
    - return  `[[TN, FP], 
   [FN, TP]]`
   
- how to flatten confusion_matrix
    - numpy.ravel() => https://rfriend.tistory.com/349
    - numpy.flatten()

```
mat = multilabel_confusion_matrix(ytrue, yhat)

TN, FP, FN, TP = mat.ravel(order='C')
```

In [3]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [144]:
yhat = model.predict(X_test)

2/2 [==============================] - 0s 13ms/step


In [145]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [147]:
accuracy_score(ytrue, yhat)

0.9230769230769231

# 11. Test in Real Time
- 웹캠을 활용한 실시간 테스트

In [83]:
from scipy.stats import rankdata

array = np.array([1,8,5,7,9])

ranks = rankdata(array)
print(array)
print(ranks)
print(np.where(ranks==4)[0][0])
print(*range(26, 23, -1))
def top3(array):
    ranks = rankdata(array)
    top3_idx = []
    for i in range(26, 23, -1):
        top3_idx.append(np.where(ranks==i)[0][0])
    return top3_idx

[1 8 5 7 9]
[1. 4. 2. 3. 5.]
1
26 25 24


In [95]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            sentence = []
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
#             predictions.append(np.argmax(res))
            predictions = top3(res)
            for i in range(3):
                sentence.append(f'{actions[predictions[i]]}: {round(res[predictions[i]] * 100, 1)}')
            print(sentence)
            
        #3. Viz logic
#             if np.unique(predictions[-10:])[0]==np.argmax(res):
#             if predictions[0] == np.argmax(res):
#                 if res[np.argmax(res)] > threshold:
#                     if len(sentence) > 0: 
#                         if actions[np.argmax(res)] != sentence[-1]:
#                             sentence.append(actions[np.argmax(res)])
#                     else:
#                         sentence.append(actions[np.argmax(res)])

#             if len(sentence) > 5: 
#                 sentence = sentence[-5:]

            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
['m: 99.8', 'b: 0.1', 'e: 0.0']
1/1 [==============================] - 0s 25ms/step
['m: 99.9', 'b: 0.1', 'e: 0.0']
1/1 [==============================] - 0s 23ms/step
['m: 100.0', 'b: 0.0', 'e: 0.0']
1/1 [==============================] - 0s 40ms/step
['m: 99.9', 'b: 0.0', 'e: 0.0']
1/1 [==============================] - 0s 25ms/step
['m: 99.9', 'e: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 27ms/step
['m: 100.0', 'e: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 30ms/step
['m: 100.0', 'e: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 23ms/step
['m: 100.0', 'e: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 27ms/step
['m: 100.0', 'e: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 24ms/step
['m: 100.0', 'e: 0.0', 'a: 0.0']
1/1 [==============================] - 0s 28ms/step
['m: 100.0', 'e: 0.0', 'a: 0.0']
1/1 [==============================] - 0s 23ms/step
['m: 100.0', 'e: 

1/1 [==============================] - 0s 24ms/step
['j: 92.7', 'n: 5.5', 't: 1.2']
1/1 [==============================] - 0s 23ms/step
['j: 99.9', 'n: 0.0', 't: 0.0']
1/1 [==============================] - 0s 23ms/step
['j: 100.0', 't: 0.0', 'm: 0.0']
1/1 [==============================] - 0s 23ms/step
['j: 100.0', 'a: 0.0', 't: 0.0']
1/1 [==============================] - 0s 26ms/step
['j: 99.8', 'a: 0.1', 'k: 0.1']
1/1 [==============================] - 0s 24ms/step
['j: 96.5', 'a: 2.2', 'k: 1.2']
1/1 [==============================] - 0s 25ms/step
['j: 72.6', 'a: 15.5', 'k: 11.6']
1/1 [==============================] - 0s 27ms/step
['a: 38.4', 'k: 34.5', 'j: 25.9']
1/1 [==============================] - 0s 24ms/step
['a: 53.3', 'k: 38.2', 'j: 5.3']
1/1 [==============================] - 0s 32ms/step
['a: 75.3', 'k: 17.3', 'i: 2.8']
1/1 [==============================] - 0s 24ms/step
['a: 91.8', 'i: 6.4', 's: 0.8']
1/1 [==============================] - 0s 23ms/step
['a: 96.2', 'm: 

1/1 [==============================] - 0s 26ms/step
['s: 54.1', 'a: 33.8', 'i: 11.1']
1/1 [==============================] - 0s 23ms/step
['s: 55.9', 'a: 29.0', 'i: 13.7']
1/1 [==============================] - 0s 24ms/step
['s: 48.8', 'a: 33.9', 'i: 15.7']
1/1 [==============================] - 0s 25ms/step
['s: 48.6', 'a: 35.5', 'i: 13.9']
1/1 [==============================] - 0s 22ms/step
['s: 58.5', 'a: 29.6', 'i: 10.0']
1/1 [==============================] - 0s 24ms/step
['s: 64.7', 'a: 27.1', 'i: 6.1']
1/1 [==============================] - 0s 22ms/step
['s: 71.2', 'a: 22.7', 'i: 3.7']
1/1 [==============================] - 0s 23ms/step
['s: 71.5', 'a: 23.5', 'i: 2.1']
1/1 [==============================] - 0s 25ms/step
['s: 83.2', 'a: 12.4', 't: 1.3']
1/1 [==============================] - 0s 28ms/step
['s: 87.4', 'a: 8.6', 't: 1.5']
1/1 [==============================] - 0s 24ms/step
['s: 87.0', 'a: 8.8', 't: 1.8']
1/1 [==============================] - 0s 23ms/step
['s: 85.9'

1/1 [==============================] - 0s 23ms/step
['i: 70.0', 't: 14.6', 's: 9.2']
1/1 [==============================] - 0s 22ms/step
['i: 60.7', 'j: 27.6', 'a: 4.9']
1/1 [==============================] - 0s 25ms/step
['j: 41.2', 'i: 29.1', 'a: 28.3']
1/1 [==============================] - 0s 25ms/step
['a: 75.8', 'j: 15.9', 'i: 7.9']
1/1 [==============================] - 0s 25ms/step
['a: 96.1', 'j: 2.4', 'i: 1.3']
1/1 [==============================] - 0s 23ms/step
['a: 97.1', 'i: 1.3', 'j: 1.3']
1/1 [==============================] - 0s 25ms/step
['a: 96.3', 'i: 2.9', 'j: 0.3']
1/1 [==============================] - 0s 27ms/step
['a: 92.7', 'i: 6.8', 's: 0.3']
1/1 [==============================] - 0s 31ms/step
['a: 94.4', 'i: 5.1', 's: 0.2']
1/1 [==============================] - 0s 23ms/step
['m: 69.6', 'a: 30.0', 'e: 0.3']
1/1 [==============================] - 0s 25ms/step
['m: 99.9', 'a: 0.0', 'e: 0.0']
1/1 [==============================] - 0s 25ms/step
['m: 100.0', 'a: 0

1/1 [==============================] - 0s 28ms/step
['i: 50.3', 'a: 49.6', 's: 0.1']
1/1 [==============================] - 0s 27ms/step
['i: 50.2', 'a: 49.7', 's: 0.1']
1/1 [==============================] - 0s 23ms/step
['i: 51.7', 'a: 48.1', 's: 0.1']
1/1 [==============================] - 0s 37ms/step
['i: 50.5', 'a: 49.4', 's: 0.1']
1/1 [==============================] - 0s 29ms/step
['a: 52.8', 'i: 47.0', 's: 0.1']
1/1 [==============================] - 0s 25ms/step
['a: 50.3', 'i: 49.5', 's: 0.1']
1/1 [==============================] - 0s 24ms/step
['i: 54.9', 'a: 44.9', 's: 0.1']
1/1 [==============================] - 0s 27ms/step
['i: 52.1', 'a: 47.7', 's: 0.2']
1/1 [==============================] - 0s 28ms/step
['a: 54.2', 'i: 45.5', 's: 0.2']
1/1 [==============================] - 0s 26ms/step
['a: 51.5', 'i: 48.3', 's: 0.2']
1/1 [==============================] - 0s 26ms/step
['i: 58.4', 'a: 41.4', 's: 0.2']
1/1 [==============================] - 0s 23ms/step
['i: 60.6', '

...

1/1 [==============================] - 0s 24ms/step
['z: 99.3', 'd: 0.3', 'r: 0.1']
1/1 [==============================] - 0s 22ms/step
['z: 99.6', 'd: 0.1', 'l: 0.1']
1/1 [==============================] - 0s 24ms/step
['z: 99.5', 'd: 0.1', 'l: 0.1']
1/1 [==============================] - 0s 23ms/step
['z: 91.5', 'r: 2.7', 'y: 2.1']
1/1 [==============================] - 0s 23ms/step
['y: 30.9', 'z: 23.5', 't: 10.2']
1/1 [==============================] - 0s 24ms/step
['i: 45.7', 'a: 13.3', 'e: 11.8']
1/1 [==============================] - 0s 25ms/step
['i: 93.5', 'a: 3.0', 'e: 1.1']
1/1 [==============================] - 0s 23ms/step
['i: 98.7', 't: 0.6', 'a: 0.4']
1/1 [==============================] - 0s 23ms/step
['i: 99.1', 't: 0.6', 'a: 0.3']
1/1 [==============================] - 0s 26ms/step
['i: 97.2', 'a: 2.4', 't: 0.3']
1/1 [==============================] - 0s 23ms/step
['i: 70.5', 'a: 28.9', 's: 0.2']
1/1 [==============================] - 0s 24ms/step
['i: 65.0', 'a: 27.

1/1 [==============================] - 0s 23ms/step
['r: 38.9', 's: 37.6', 'v: 9.0']
1/1 [==============================] - 0s 26ms/step
['r: 45.2', 's: 29.5', 'v: 9.6']
1/1 [==============================] - 0s 30ms/step
['r: 45.8', 's: 28.5', 'v: 9.5']
1/1 [==============================] - 0s 36ms/step
['r: 42.9', 's: 29.7', 'v: 9.7']
1/1 [==============================] - 0s 23ms/step
['r: 37.4', 's: 35.6', 'a: 9.4']
1/1 [==============================] - 0s 22ms/step
['s: 41.3', 'r: 32.3', 'a: 10.4']
1/1 [==============================] - 0s 26ms/step
['s: 46.9', 'r: 27.1', 'a: 11.7']
1/1 [==============================] - 0s 24ms/step
['s: 52.5', 'r: 21.6', 'a: 13.3']
1/1 [==============================] - 0s 24ms/step
['s: 55.9', 'r: 18.0', 'a: 15.0']
1/1 [==============================] - 0s 25ms/step
['s: 58.1', 'a: 16.9', 'r: 14.0']
1/1 [==============================] - 0s 23ms/step
['s: 59.7', 'a: 17.4', 'r: 8.3']
1/1 [==============================] - 0s 23ms/step
['s: 57.

1/1 [==============================] - 0s 28ms/step
['v: 92.6', 's: 3.4', 'x: 2.7']
1/1 [==============================] - 0s 23ms/step
['v: 97.2', 's: 1.0', 'x: 0.9']
1/1 [==============================] - 0s 36ms/step
['v: 98.6', 's: 0.4', 'u: 0.4']
1/1 [==============================] - 0s 26ms/step
['v: 99.2', 'u: 0.3', 's: 0.2']
1/1 [==============================] - 0s 26ms/step
['v: 99.4', 'u: 0.3', 'l: 0.1']
1/1 [==============================] - 0s 27ms/step
['v: 99.6', 'u: 0.3', 'l: 0.0']
1/1 [==============================] - 0s 34ms/step
['v: 99.7', 'u: 0.2', 'l: 0.0']
1/1 [==============================] - 0s 32ms/step
['v: 99.7', 'u: 0.2', 'l: 0.0']
1/1 [==============================] - 0s 26ms/step
['v: 99.8', 'u: 0.1', 'x: 0.0']
1/1 [==============================] - 0s 28ms/step
['v: 99.8', 'u: 0.1', 'x: 0.0']
1/1 [==============================] - 0s 26ms/step
['v: 99.9', 'u: 0.1', 'x: 0.0']
1/1 [==============================] - 0s 29ms/step
['v: 99.9', 'u: 0.1', 'x

1/1 [==============================] - 0s 23ms/step
['w: 100.0', 'r: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 29ms/step
['w: 100.0', 'r: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 26ms/step
['w: 100.0', 'r: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 25ms/step
['w: 100.0', 'r: 0.0', 'b: 0.0']
1/1 [==============================] - 0s 25ms/step
['w: 100.0', 'r: 0.0', 'q: 0.0']
1/1 [==============================] - 0s 24ms/step
['w: 100.0', 'r: 0.0', 'q: 0.0']
1/1 [==============================] - 0s 24ms/step
['w: 100.0', 'r: 0.0', 'q: 0.0']
1/1 [==============================] - 0s 23ms/step
['w: 93.5', 'q: 3.5', 'r: 3.0']
1/1 [==============================] - 0s 23ms/step
['q: 64.6', 'r: 25.2', 'w: 9.4']
1/1 [==============================] - 0s 24ms/step
['q: 59.6', 'r: 18.9', 'z: 17.0']
1/1 [==============================] - 0s 24ms/step
['z: 61.7', 'q: 30.4', 't: 2.8']
1/1 [==============================] - 0s 27ms/step
['q: 63.1', '

1/1 [==============================] - 0s 24ms/step
['z: 100.0', 't: 0.0', 'l: 0.0']
1/1 [==============================] - 0s 25ms/step
['z: 100.0', 't: 0.0', 'q: 0.0']
1/1 [==============================] - 0s 23ms/step
['z: 100.0', 'q: 0.0', 't: 0.0']
1/1 [==============================] - 0s 23ms/step
['z: 100.0', 'q: 0.0', 't: 0.0']
1/1 [==============================] - 0s 33ms/step
['z: 100.0', 't: 0.0', 'q: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 100.0', 't: 0.0', 'd: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 100.0', 't: 0.0', 'd: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 100.0', 't: 0.0', 'l: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 100.0', 'l: 0.0', 't: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 100.0', 'l: 0.0', 't: 0.0']
1/1 [==============================] - 0s 24ms/step
['z: 99.7', 'l: 0.3', 'm: 0.0']
1/1 [==============================] - 0s 35ms/step
['z: 99.0', 'l